In [ ]:
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# sparkSession = spark --> in the case of EMR
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")

In [ ]:
from pyspark.sql.types import *

# hours between 06:00 and 22:59:59
for day in range(1,32):
    traces = spark.read.parquet("s3a://mobility-traces-sp/processed-data/records_between-6-23-with-all-regions/MO_1510"+ str(day) + "/")
    traces_in_sp = traces.filter('region != "None"')
    traces_out_sp = traces.filter('region == "None"')
    traces_in_sp.repartition(60).write.parquet("s3://mobility-traces-sp/processed-data/records_between-6-23-only-in-sp-regions/MO_1510"+ str(day) + "/")
    traces_out_sp.repartition(60).write.parquet("s3://mobility-traces-sp/processed-data/records_between-6-23-out-of-sp-region/MO_1510"+ str(day) + "/")